# Лабораторная работа 3. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы БВТ2104 Юдин Артём 
***

### Задание

1.  Реализовать алгоритмы поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



### Выполнение:

In [12]:
import numpy as np
from copy import deepcopy
from collections import deque

In [13]:
class Graph:
    def __init__(self, vertices:int) -> None:
        self.V = vertices
        self.graph = []

    def add_edge(self, u:int, v:int, w:int) -> None:
        self.graph.append([u, v, w])

    def delete_edge(self):
        self.graph.pop()
        

    def print_graph(self, dist:list[int], p:list[int], f:int) -> None:
        path = []
        path.append(f)
        next_key = p[f]

        while next_key:
            path.append(next_key)
            next_key = p[next_key]

        path.append(next_key)
        path.reverse()

        print("Distances from the source:")
        
        for i in range(len(dist)):
            print(f"Vertice: {i}, Distance: {dist[i]}")

        print("Path: ", path)

        

In [14]:
g = Graph(4)
g.add_edge(0, 1, 10)
g.add_edge(0, 2, -6)
g.add_edge(1, 2, 7)
g.add_edge(1, 3, 53)
g.add_edge(2, 1, -2)
g.add_edge(2, 3, 6)

### Алгоритм Флойда-Уоршелла

Проходимся по всему графу и заносим все значения в массив, а также при хождение более короткого маршрута заменяем значения родителей. Сложность в V раз больше Дейкстре = O(v^3)

In [15]:
def floyd_warshall(gr, src:int, fin:int) -> None:
    distance = np.array([np.inf] * gr.V * gr.V).reshape(gr.V, gr.V)
    pred = np.array([-1] * gr.V * gr.V).reshape(gr.V, gr.V)
    for i in range(gr.V):
        for j in range(gr.V):
            if i == j:
                distance[i][j] = 0
                pred[i][j] = 0
            
    for a, b, c in gr.graph:
        distance[a][b] = c
        pred[a][b] = b
    
    for k in range(gr.V):
        for i in range(gr.V):
            for j in range(gr.V):
                
                #проверяем, не нашли ли мы более короткий путь
                if distance[i][j] > distance[i][k] + distance[k][j]:
                    distance[i][j] = distance[i][k] + distance[k][j]
                    pred[i][j] = k #новый родитель

    print(distance)
    print(get_path_floyd_warshall(pred, src, fin))

#восстанавливаем путь
def get_path_floyd_warshall(P, u:int, v:int) -> list[int]:
    path = [u]

    while u != v:
        u = P[u][v]
        path.append(u)

    return path

### Алгоритм Дейкстры

Алгоритм поиска кратчайшего пути между 2 вершинами взвешенного графа O((V + E) * log(V)) 

*ещё есть реализация для плотных графов O(V^2 + E)

In [16]:
def dijkstra(gr, src:int) -> None:
    map_heap = {}

    for i in range(gr.V):
        map_heap[i] = np.inf

    map_heap.pop(src)
    distance = [np.inf] * gr.V
    distance[src] = 0
    pred = [-1] * gr.V
    pred[src] = None

    for a, b, c in gr.graph:
        if a == src:
            map_heap[b] = c
            pred[b] = src

    while map_heap:
        smallest = min(map_heap.values())
        temp_key = list(map_heap.keys())[list(map_heap.values()).index(smallest)]
        distance[temp_key] = map_heap.pop(temp_key)
        
        for a, b, c in gr.graph:
            if a == temp_key and b in map_heap.keys():
                if map_heap[b] > distance[temp_key] + c:
                    map_heap[b] = distance[temp_key] + c
                    pred[b] = temp_key

    gr.print_graph(distance, pred, 3)

### Алгоритм Беллмана-Форда

In [17]:
def bellman_ford(gr:list[int], src:int) -> None:
        
    distance = [np.inf] * gr.V
    distance[src] = 0
    pred = [-1] * gr.V
    pred[src] = None

    for _ in range(gr.V - 1):
        for a, b, c in gr.graph:
            if distance[a] != np.inf and distance[a] + c < distance[b]:
                distance[b] = distance[a] + c
                pred[b] = a

    for a, b, c in gr.graph:
        if distance[a] != np.inf and distance[a] + c < distance[b]:
            print("Graph contains negative weight cycle")

            return

    gr.print_graph(distance, pred, 3)
    
    return distance

### Алгоритм Джонсона

In [18]:
def johnson(gr, src):
    new_graph = deepcopy(gr.graph)
    
    for i in range(gr.V):
        gr.add_edge(gr.V, i, 0)

    gr.V += 1
    h = bellman_ford(gr, gr.V -1)
    h.pop(-1)
    gr.V -= 1
    gr.graph = deepcopy(new_graph)
    gr.graph = []

    for a,b,c in new_graph:
        new_w = c + h[a] - h[b]
        gr.add_edge(a, b, new_w)

    print(gr.graph)
    dijkstra(gr, src)


In [19]:
johnson(g, 0)

Distances from the source:
Vertice: 0, Distance: 0
Vertice: 1, Distance: -8
Vertice: 2, Distance: -6
Vertice: 3, Distance: 0
Vertice: 4, Distance: 0
Path:  [None, 4, 3]
[[0, 1, 18], [0, 2, 0], [1, 2, 5], [1, 3, 45], [2, 1, 0], [2, 3, 0]]
Distances from the source:
Vertice: 0, Distance: 0
Vertice: 1, Distance: 0
Vertice: 2, Distance: 0
Vertice: 3, Distance: 0
Path:  [0, 2, 3]


### Алгоритм Левита

In [20]:
def levit(gr, src):
    m0 = []
    m1 = deque()
    m2 = [x for x in range(gr.V)]
    m1.append(m2.pop(src))
    dist = [np.inf for _ in range(gr.V)]
    dist[src] = 0
    pred = [-1 for _ in range(gr.V)]
    pred[src] = None
    
    while m1:
        current = m1.popleft()

        for a, b, c in gr.graph:
            if a == current:
                if b in m2:
                    dist[b] = dist[a] + c
                    m2.remove(b)
                    m1.append(b)
                    pred[b] = a
                
                elif b in m1:
                    if dist[b] >= dist[a] + c:
                        dist[b] = dist[a] + c
                        pred[b] = a

                elif b in m0:
                    if dist[b] > dist[a] + c:
                        dist[b] = dist[a] + c
                        pred[b] = a
                        m0.remove(b)
                        m1.appendleft(b)
        
        m0.append(current)
        
    gr.print_graph(dist, pred, 3)

### Алгоритм Йена

In [21]:
def yen(gr, src):
    

IndentationError: expected an indented block (2529190076.py, line 2)

### Вывод